#### passing parameters to the notebook (adding a widget)

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

#### invoke a notebook from another notebook using the %run command

In [0]:
%run "../Includes/configuration"

In [0]:
%run  "../Includes/common_functions"

In [0]:
pit_stops_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                     StructField("driverId", IntegerType(), True),
                                     StructField("stop", IntegerType(), True),
                                     StructField("lap", IntegerType(), True),
                                     StructField("time", StringType(), True),
                                     StructField("duration", StringType(), True),
                                     StructField("milliseconds", StringType(), True)])

In [0]:
pit_stops_df = spark.read \
.option("multiLine", True)\
.schema(pit_stops_schema) \
.json(f"{raw_folder_path}/pit_stops.json")

In [0]:
pit_stops_final_df = pit_stops_df.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumn("ingestion_date", current_timestamp())\
.withColumn("pit_stops_id", col("stop").cast("integer"))\
.withColumn("data_source", lit(v_data_source))


In [0]:
pit_stops_final_df= pit_stops_final_df.drop("ingestion_date")

In [0]:
pit_stops_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/pit_stops")

In [0]:
dbutils.notebook.exit("Success")